# **ch.08 이미지를 위한 인공 신경망**  
## 08-1 합성곱 신경망의 구성 요소  
### 풀어야 할 문제:
> ### Fashion MNIST classification(image classification)에서 더 좋은 performance를 보여주는 convolution neural network에 대해 알아보라

<br />

### *합성곱*  
convolution은 마치 input data에 도장을 찍어 유용한 feature만 드러나게 하는 것으로 비유할 수 있음  

<br />

앞서 model을 만들 때 사용한 dense layer에는 neuron마다 input 개수 만큼의 weights가 있음  
&rarr; 모든 input에 weight를 곱함

<br />

neural network는 처음에 weights와 intercepts를 random하게 초기화한 다음 epoch를 반복하며 gradinet descent algorithm을 사용하여 loss가 낮아지도록 최적의 weights와 intercepts를 찾아감  
&rarr; training의 과정

<br />

dense layer의 output의 수는 해당 layer의 neuron 수와 같음  
이는 input의 size와 상관없음

<br />

convolution은 dense layer의 계산과 다름  
input data 전체에 weights를 곱하는 것이 아니라 일부에 weights를 곱함  
예를 들어 neuron의 weight $w_1$ ~ $w_3$가 입력의 처음 3개 feature와 곱해져 1개의 output을 만들고, 같은 neuron이 한 칸 이동하여 두 번째부터 네 번째 3개의 feature와 곱해져 새로운 output을 만듦  
&rarr; 이와 같은 과정이 반복되어 output을 만드는 것이 convolution  
이 때 중요한 것은 첫 번째 convolution 연산에 사용된 neuron($w_1$ ~ $w_3$, intercept)이 계속해서 동일하게 사용된다는 것

<br />

dense layer의 neuron은 input 개수만큼 weights를 가지고 1개의 output을 만듦  
convolution layer의 neuron은 일부 weights를 가지고 여러 개의 output을 만듦  
(convolution layer의 nueron의 weights 개수는 hyperparameter)  
&rarr; 이러한 과정은 마치 input data 위를 이동하며 도장을 찍는 것처럼 생각할 수 있음  
&rarr; 도장을 찍을 때마다 output이 하나씩 만들어짐

<br />

이러한 convolution layer의 neuron은 그 역할이나 작동하는 방식이 dense layer의 neuron과 많이 다르고 dense layer의 표현 방식으로 나타내기 어렵기 때문에 filter 혹은 kernel이라고 함  
keras API에서는 neuron 개수를 이야기할 때 filter라 하고, weights를 이야기 할 때는 kernel이라 부름

<br />

input이 2 dimension array이면 filter도 2 dimension이어야 함  
convolution layer에서 filter는 왼쪽 위에서부터 오른쪽으로 이동하며 convolution 연산을 수행하며, 모든 input을 빼놓지 않음  
convolution 연산은 input과 weights의 row와 column을 맞추어 곱하고 모두 더한 후, intercept를 더하여 output을 만드는 것  
convolution의 output 개수는 filter가 이동한 횟수와 같음  
&rarr; convolution 연산을 수행한 횟수와 같음


이 때 output은 filter가 input을 지나간 자리에 맞게 2 dimension으로 배치됨  
예를 들어, 첫 번째 output은 2 dimension array에서 가장 왼쪽 위(0, 0)에 배치됨  
&rarr; input을 압축한 듯한 모양  
이러한 convolution의 output을 feature map이라 함

<br />

dense layer에서 여러 개의 neuron을 사용하듯, convolution layer에서도 여러 개의 filter를 사용함  
여러 개의 filter를 사용하면 만들어진 feature map들은 순서대로 쌓임  
&rarr; 2 dimension array인 feature map을 여러개 쌓았으니 3 dimension이 됨


dense layer의 neuron의 weights가 모두 다르듯, convolution layer의 weights(kernel)도 모두 다름

<br />

이처럼 convolution은 2 dimension을 그대로 활용하기 때문에 convolution neural network는 image processing에서 뛰어난 performance를 보여줌

<br />

- - -

<br />

### *케라스 합성곱 층*  
keras는 다른 layer와 마찬가지로 convolution layer도 `keras.layers` package 아래에 class로 제공  
특별히 앞서 살펴본 방식으로 작동하는(왼쪽 위부터 오른쪽으로) convolution layer는 `Conv2D` class로 제공


`Conv2D` class의 첫 번째 parameter는 filter의 개수  
`kernel_size` parameter로 filter에 사용할 kernel의 size를 지정  
&rarr; 이 두 parameter는 반드시 지정해야 하는 parameter  
dense layer처럼 `activation` parameter로 activation function을 지정

<br/>

convolution neural network: 1개 이상의 convolution layer를 사용한 neural network  
&rarr; convolution layer만 사용해야만 convolution neural network라고 하는 것이 아님


실제로 class에 대한 probability를 계산하기 위해 마지막 layer에 class 개수만큼의 neuron을 가진 dense layer를 두는 것이 일반적

<br/>

#### 패딩과 스트라이드  
padding
+ filter나 kernel을 조정하지 않고 output의 size를 조절하기 위해 input data 주위를 가상의 원소(0)으로 채우는 것  
&rarr; filter가 지날 수 있는 영역을 늘려 지나는 횟수를 늘려 output을 조절
+ 종류  
  + same padding:  
  input과 feature map의 size를 동일하게 만들기 위해 input 주위를 0으로 padding하는 것
  + valid padding:  
  padding 없이 순수한 input array만 convolution 연산을 진행하여 feature map을 만드는 것  
  &rarr; valid padding은 feature map의 크기가 줄어들 수 밖에 없음
+ 사용하는 이유  
padding 없이 convolution 연산을 진행한다면 filter가 지나는 횟수가 적은 특정 data가 다른 data에 비해 상대적으로 filter에 소외되는 경우가 발생  
&rarr; 중요한 feature를 포함하는 data가 소외되어 중요한 information이 feature map에 전달되지 않을 수 있음


적절한 padding은 information을 잃어버리지 않도록 도와줌  
일반적인 convolution neural network에서는 same padding이 많이 사용됨


`Conv2D` class의 `padding` parameter에 `'valid'` 혹은 `'same'`을 전달하여 padding을 지정할 수 있음

<br/>

stride
+ filter가 이동하는 간격의 크기  
&rarr; stride가 1이면 한 칸씩 이동하며 convolution 연산 수행
+ `Conv2D` class의 `stride` parameter는 horizontal 이동 방향과 vertical 이동 방향에 대한 크기를 tuple로 전달할 수 있음  
그러나 horizontal과 vertical 방향을 다르게 지정하는 경우는 거의 없음
+ 1이상의 stride를 사용하는 경우는 드뭄

<br/>

#### 풀링  
pooling
+ convolution layer에서 만든 feature map의 size를 줄이는 것
+ feature map의 개수는 줄이지 않음  
&rarr; kernel이 없는 filter를 적용하는 것과 비슷한 효과
+ 동작  
  + filter처럼 위를 지나가며 연산을 수행
  + convolution과 달리 feature map 위 data를 겹치지 않고 움직임
  + filter와 달리 weights가 없음
  + feature map 위를 지나며 maximum을 찾거나, mean을 구함
+ 종류
  + max pooling:  
  maximum을 구하는 연산을 수행하는 pooling
  + average pooling:  
  해당 구역의 mean을 구하는 pooling
  + 많은 경우 max pooling을 많이 사용  
  $\because$ average pooling은 feature map에 있는 중요한 information을 희석시킬 수 있기 때문
+ `keras.layers` package 아래 `MaxPooling2D` class 와 `AvragePooling2D` class로 구현
  + `MaxPooling2D` class:
    + 첫 번째 parameter로 pooling의 size 지정
    + `strides`와 `padding` parameter로 stride와 padding을 지정할 수 있지만, 대부분의 경우 기본값 그대로 사용
  + `AveragePooling2D` class:
    + 첫 번째 parameter로 pooling의 size 지정
    + `strides`와 `padding` parameter로 stride와 padding을 지정할 수 있지만, 대부분의 경우 기본값 그대로 사용
+ 사용하는 이유
  + feature map의 크기는 convolution의 stride를 조정하여 조절할 수 있으나 pooling을 사용하는 것이 권장됨  
  $\because$ stride를 크게하여 feature map의 size를 줄이는 것보다 pooling을 사용하는 것이 경험적으로 좋은 performance를 보여주기 때문

<br/>

- - -

<br/>

### *합성곱 신경망의 전체 구조*  

input data &rarr; padding &rarr; convolution &rarr; activation function &rarr; feature map &rarr; pooling &rarr; flatten &rarr; output layer &rarr; prediction

<br/>

#### 컬러 이미지를 사용한 합성곱  
지금까지는 input data를 2 dimension이라 가정함  
이는 grayscale image sample을 가진 Fashion MNIST에는 적합하지만, 색을 가진 color image는 2 dimension으로 표현할 수 없기에 적절하지 않음  
color image는 RGB channel도 포함하기에 3 dimension array로 표현  
&rarr; 하나의 color image는 length, width, channel(depth))의 3 dimension array

<br/>

이러한 3 dimension data에 convolution을 적용하기 위해서는 filter에도 depth가 있어야 함  
&rarr; 3 dimension filter가 필요함


input data와 filter가 3 dimension이라고 해도 만들어지는 output은 1개임  
&rarr; feature map은 2 dimension


keras의 convolution layer는 항상 3 dimension의 input을 기대함  
만약 grayscale의 2 dimension data일 경우에는 channel이 1인 3 dimension array로 변환하여 전달함  
이와 같이 변환을 수행하는 경우는 더 있는데, 바로 convolution layer - pooling layer 다음에 또 다시 convolution layer가 이어질 때임


이렇게 convolution layer가 여러 개일 경우, 각각의 feature map의 size는 줄어들고 depth는 깊어짐

<br/>

convolution neural network에서 filter는 image에 있는 어떤 특징 찾는다고 생각할 수 있음  
처음에는 간단하고 추상적인 특징을 찾고 layer가 깊어질수록 다양하고 구체적인 특징을 감지할 수 있도록 filter의 개수를 늘림  
또한 어떤 특징이 image의 어느 위치에 놓이더라도 쉽게 감지할 수 있도록 length와 width dimension을 점점 줄여가는 것